In [ ]:
import torch
import numpy as np
import pandas as pd
import random # 시드 고정을 위해
import os # 시드 고정을 위해
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from tqdm.auto import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

모델 학습에 사용할 cpu or gpu 장치 확인

In [ ]:
SEED = 42
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# 재현성 함수(Reproduction)

In [ ]:
def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)    # 파이썬 환경변수 시드 고정
    np.random.seed(seed)
    torch.manual_seed(seed) # cpu 연산 무작위 고정
    torch.cuda.manual_seed(seed) # gpu 연산 무작위 고정
    torch.backends.cudnn.deterministic = True  # cuda 라이브러리에서 Deterministic(결정론적)으로 예측하기 (예측에 대한 불확실성 제거 )

# 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 구글 드라이브 데이터 경로(수정하시오)

In [ ]:
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/03.MachineLearning/data/"

# 학습및 테스트 데이터

In [ ]:
train_churn = pd.read_csv(f"{DATA_PATH}train_churn.csv")
test_churn = pd.read_csv(f"{DATA_PATH}test_churn.csv")
train_churn.shape ,test_churn.shape

((5282, 21), (1761, 20))

# 결측치 확인

In [ ]:
train_churn.isnull().sum().sum() , test_churn.isnull().sum().sum()

(0, 0)

# 학습및 정답 데이터와 추론데이터 전처리

- 수치형 변수 

In [ ]:
train = train_churn.select_dtypes("number").iloc[:,:-1] # 정답값 제외
target = train_churn.iloc[:,-1] # 정답 데이터
test = test_churn.select_dtypes("number") # 추론 데이터

train.shape, test.shape , target.shape

((5282, 17), (1761, 17), (5282,))

- 범주형 변수 원핫 인코딩

In [ ]:
cols = ["InternetService","PaymentMethod"]
enc = OneHotEncoder(handle_unknown="ignore")

# 학습데이터
tmp = pd.DataFrame(
    enc.fit_transform(train_churn[cols]).toarray(),
    columns = enc.get_feature_names_out()
)

train = pd.concat([train,tmp],axis=1)

# 추론 데이터
tmp = pd.DataFrame(
    enc.transform(test_churn[cols]).toarray(),
    columns = enc.get_feature_names_out()
)

test = pd.concat([test,tmp],axis = 1)

train.shape, test.shape

((5282, 24), (1761, 24))

# 스케일링

In [ ]:
scaler = MinMaxScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

train.shape, test.shape , target.mean()

((5282, 24), (1761, 24), 0.2631578947368421)

In [ ]:
train

array([[0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [ ]:
# from sklearn.model_selection import train_test_split
# x_train, x_valid, y_train, y_valid = train_test_split(features, target, random_state=SEED)
# x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

# 여기서 부터 pytorch 를 이용하여  학습및 테스트 데이터에대한 확률을 추론해서 제출해 주세요.

In [ ]:
class ChurnDataset(torch.utils.data.Dataset):
    def __init__(self,x,y=None): # 정답값이 없는 경우를 가정해서 디폴트 파라미터 `None`을 정의 
        self.x = x
        self.y = y
        if self.y is not None:
            self.y = y.astype("float32").values.reshape(-1,1)

    def __len__(self): 
        return self.x.shape[0]

    def __getitem__(self,idx): 
        item = {} # 딕셔너리로 넣어보자
        item["x"] = torch.Tensor(self.x[idx])

        if self.y is not None:
            item["y"] = torch.Tensor(self.y[idx])

        return item

In [ ]:
train_dt = ChurnDataset(train,target)
train_dt[1]

{'x': tensor([1.0000, 0.0000, 0.0000, 0.0000, 0.0972, 1.0000, 0.0000, 0.0000, 0.0000,
         1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5670, 0.0605, 0.0000,
         1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000]), 'y': tensor([0.])}

In [ ]:
tr_dl = torch.utils.data.DataLoader(train_dt,batch_size=2,shuffle=False)
next(iter(tr_dl))

{'x': tensor([[0.0000, 0.0000, 0.0000, 1.0000, 0.0139, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0703, 0.0029, 1.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000],
         [1.0000, 0.0000, 0.0000, 0.0000, 0.0972, 1.0000, 0.0000, 0.0000, 0.0000,
          1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5670, 0.0605, 0.0000,
          1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000]]), 'y': tensor([[1.],
         [0.]])}

In [ ]:
#

In [ ]:
train.shape[1] #입력값

24

In [ ]:
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

input_layer = torch.nn.Linear(train.shape[1],1)
data = next(iter(tr_dl))

hidden_layer = input_layer(data["x"])
hidden_layer

tensor([[-0.0285],
        [-0.3030]], grad_fn=<AddmmBackward0>)

In [ ]:
#

In [ ]:
class Net(torch.nn.Module):
    def __init__(self,in_features):
        super().__init__()
        self.seq = torch.nn.Sequential(
            torch.nn.Linear(in_features,8),
            torch.nn.ELU(), # 이 층에대한 비선형성
            torch.nn.Linear(8,6),
            torch.nn.ELU(),
            torch.nn.Linear(6,4),
            torch.nn.ELU(),
            torch.nn.Linear(4,2),
            torch.nn.ELU(),
            torch.nn.Linear(2,1),
        )
    def forward(self,x):
        x = self.seq(x)
        return x

In [ ]:
model = Net(train.shape[1])
model(data["x"])

tensor([[-0.0475],
        [-0.0261]], grad_fn=<AddmmBackward0>)

In [ ]:
batch_size = 32 # 미니 배치 사이즈
loss_fn = torch.nn.BCEWithLogitsLoss()
model = Net(train.shape[1]).to(device) # 열부분 => in_features / GPU 연결
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

tr_dt = ChurnDataset(train, target)
tr_dl = torch.utils.data.DataLoader(tr_dt,batch_size=batch_size,shuffle=True)

x_test = test.copy() 
test_dt = ChurnDataset(x_test)
test_dl = torch.utils.data.DataLoader(test_dt,batch_size=batch_size,shuffle=False)

epoch = 80

num_features = train.shape[1]

In [ ]:
#학습모드
model.train()

for batch in tr_dl: # batch단위로 32개씩 꺼내보다
    # print(batch["x"].shape)
    # break
    
    pred = model(batch["x"].to(device)) # 예측값
    loss = loss_fn(pred,batch["y"].to(device))# 손실값 = 손실함수(예측값 , 정답값)

    optimizer.zero_grad() # 업데이트하기 전 초기화해야한다. optimizer는 계속해서 가중치를 더하기 때문에 누적되는걸 방지하려고
    loss.backward() # 역전파
    optimizer.step() # 가중치 업데이트

    # 눈으로 확인하자
    batch_loss = loss.item()

    print(batch_loss)

In [ ]:
#학습모드
model.train()

epoch_loss = 0
for batch in tr_dl: # batch단위로 32개씩 꺼내보다
   
    pred = model(batch["x"].to(device)) # 예측값
    loss = loss_fn(pred,batch["y"].to(device))# 손실값 = 손실함수(예측값 , 정답값)

    optimizer.zero_grad() # 업데이트하기 전 초기화해야한다. optimizer는 계속해서 가중치를 더하기 때문에 누적되는걸 방지하려고
    loss.backward() # 역전파
    optimizer.step() # 가중치 업데이트

    # 눈으로 확인하자
    epoch_loss += loss.item()
epoch_loss /= len(tr_dl)

print(f"epoch_loss : {epoch_loss}")

epoch_loss : 0.46928108698991405


In [ ]:
#

In [ ]:
def train_loop(tr_dl,model,loss_fn,optimizer,device):
    #학습모드
    model.train()

    epoch_loss = 0
    for batch in tr_dl: # batch단위로 32개씩 꺼내보다
    
        pred = model(batch["x"].to(device)) # 예측값
        loss = loss_fn(pred,batch["y"].to(device))# 손실값 = 손실함수(예측값 , 정답값)

        optimizer.zero_grad() # 업데이트하기 전 초기화해야한다. optimizer는 계속해서 가중치를 더하기 때문에 누적되는걸 방지하려고
        loss.backward() # 역전파
        optimizer.step() # 가중치 업데이트

        # 눈으로 확인하자
        epoch_loss += loss.item()
    epoch_loss /= len(tr_dl)

    return epoch_loss

In [ ]:
train_loop(tr_dl,model,loss_fn,optimizer,device)

0.4357486392360136

In [ ]:
# 검증셋에 대한 평가

In [ ]:
# @torch.no_grad() # => with torch.no_grad()를 데코레이터화 # 평가할때는 가중치 업데이트 할 필요가 없기에 Gradient를 구할 필요가 없다.
# def test_loop(dataloader,model,loss_fn,device): # 평가할땐 가중치(optimizer)는 필요 없을듯?
#     epoch_loss = 0
#     model.eval() # 모델모드를 eval로 변경하기! -> 평가모드 / 모델의 랜덤적인 요소를 뜻한다.
#   # 결론. torch.no_grad()와  eval()은 다른 의미라 두가지를 꼭 같이 써야한다.
    
#     for batch in dataloader:
#         pred = model(batch["x"].to(device))
#         loss = loss_fn(pred,batch["y"].to(device))

#         epoch_loss += loss.item()

#     epoch_loss /= len(tr_dl)
#     return epoch_loss

In [ ]:
# valid_dt = ChurnDataset(x_valid,y_valid)
# valid_dl = torch.utils.data.DataLoader(valid_dt,batch_size=batch_size,shuffle=False)

In [ ]:
# test_loop(valid_dl,model,loss_fn,device) # 평가에 대한 손실점수

In [ ]:
@torch.no_grad() 
def test_loop(dataloader,model,loss_fn,device): 
    epoch_loss = 0
    model.eval() 
    
    pred_list=[]
    sig = torch.nn.Sigmoid()


    for batch in dataloader:
        pred = model(batch["x"].to(device))

        if batch.get("y") is not None:  # y값이 있을 경우만 loss 계산
            loss = loss_fn(pred,batch["y"].to(device))
            epoch_loss += loss.item()

        pred = sig(pred)    # 0 ~ 1 확률값으로 변경
        pred = pred.to("cpu").numpy() # cpu로 이동후 시그모이드 함수로 계산한 예측값을 넘파이로 변환
        pred_list.append(pred)

    epoch_loss /= len(tr_dl)
    pred = np.concatenate(pred_list)
    
    return epoch_loss, pred

In [ ]:
# test_dt = ChurnDataset(x_test)
# test_dl = torch.utils.data.DataLoader(test_dt,batch_size=batch_size,shuffle=False)

# _,pred = test_loop(valid_dl,model,loss_fn,device)
# pred

In [ ]:
# from sklearn.metrics import roc_auc_score
# roc_auc_score(y_valid,pred)

In [ ]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

In [ ]:
loss_fn = torch.nn.BCEWithLogitsLoss()
epoch = 80
batch_size = 32
num_features = train.shape[1]

In [ ]:
is_holdout = False
for i,(tri,val) in enumerate(cv.split(train)): # 5개 빌드로 나눔
    # 학습데이터
    x_train = train[tri]
    y_train = target[tri]

    # 검증데이터
    x_valid = train[val]
    y_valid = target[val]

    model = Net(num_features).to(device)
    optimizer = torch.optim.Adam(model.parameters())

    tr_dt = ChurnDataset(x_train,y_train)
    valid_dt = ChurnDataset(x_valid,y_valid)
    
    tr_dl = torch.utils.data.DataLoader(tr_dt,batch_size=batch_size, shuffle=True) # 학습데이터
    valid_dl = torch.utils.data.DataLoader(valid_dt,batch_size=batch_size, shuffle=False) # 검증데이터


    best_score = 0
    patience = 0

    for e in range(epoch):
        train_loss = train_loop(tr_dl, model, loss_fn, optimizer, device)
        valid_loss,pred = test_loop(valid_dl, model, loss_fn, device)

        score = roc_auc_score(y_valid,pred)
        patience += 1
        if best_score < score : # 작으면
            patience = 0 
            best_score = score # 갱신
            torch.save(model.state_dict(),f"model_{i}.pth") # 저장

        if patience == 5:
            break
    print(f"{i} 번째 폴드 AUC: {best_score}")

    if is_holdout:
        break

0 번째 폴드 AUC: 0.8413456328565236
1 번째 폴드 AUC: 0.8387392391002498
2 번째 폴드 AUC: 0.8496151226000194
3 번째 폴드 AUC: 0.853510108562016
4 번째 폴드 AUC: 0.8215268051888418


In [ ]:
x_test.shape # 정답이 없는 테스트셋이라 가정

(1761, 24)

In [ ]:
test_dt = ChurnDataset(x_test)
test_dl = torch.utils.data.DataLoader(test_dt,batch_size=batch_size,shuffle=False)

In [ ]:
pred_list=[]
for i in range(5):
    model = Net(num_features).to(device)
    state_dict = torch.load(f"model_{i}.pth") # 가중치
    model.load_state_dict(state_dict)
    _, pred = test_loop(test_dl,model,loss_fn,device)
    pred_list.append(pred)
pred = np.mean(pred_list, axis=0)
pred

array([[0.6514388 ],
       [0.10194989],
       [0.01590842],
       ...,
       [0.11063508],
       [0.6916496 ],
       [0.22220583]], dtype=float32)

In [ ]:
#pred = 0 # 테스트 데이터에 대한 예측 확률, AUC 평가

# 제출파일에 예측확률 넣기

In [ ]:
submit = pd.read_csv(f"{DATA_PATH}submit_churn.csv")
submit["target"] = pred
submit

,customerID,target
0,1024-GUALD,0.651439
1,0484-JPBRU,0.101950
2,3620-EHIMZ,0.015908
3,6910-HADCM,0.663701
4,8587-XYZSF,0.020692
...,...,...
1756,4581-SSPWD,0.734485
1757,4813-HQMGZ,0.696479
1758,4903-CNOZC,0.110635
1759,0822-GAVAP,0.691650


# 제출파일 구글드라이브에 저장

In [ ]:
submit.to_csv(f"{DATA_PATH}손성우_submit.csv",index=False)

# 0.8563 - dropout(0.2), epoch:100
# 0.85988 - ELU( ,16)->(16,8)->(8,4)->(4,1) epoch:1000 

In [ ]:
# private_score = 0.86000